In [1]:
from infrastructure.database.mongo_client import MongoDBClient
from infrastructure.repository import YouTubeContentRepository
from application.use_case import YouTubeContentDownloadAndStore
from application.strategy.stt_strategy_factory import STTStrategyType, STTStrategyFactory

import os
from dotenv import load_dotenv

load_dotenv()

# MongoDB 클라이언트 초기화
client = MongoDBClient(uri=os.environ['MONGO_CONNECTION_STRING'])
client.connect()

# 저장소 초기화
repository = YouTubeContentRepository(client, 'test')

# STT 초기화
stt_strategy = STTStrategyFactory.create(STTStrategyType.LOCAL_WHISPER)

# 유즈케이스 초기화
yds = YouTubeContentDownloadAndStore(repository)

INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [allow_tf32, disable_jit_profiling]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []


Connected to MongoDB


Device set to use mps


In [2]:
test_youtube_link = 'https://www.youtube.com/watch?v=MmW166cHj54'

# 테스트 오디오 파일 다운로드
yds.execute(test_youtube_link, 'test', '../downloads')

데이터가 이미 존재합니다.
../downloads/test
파일이 이미 존재합니다: ../downloads/test/유니티 뱀서라이크 개발강좌가 온다?! 언데드 서바이버.mp3


True

In [3]:
contents = repository.find_without_script()
print(contents[0])

YouTubeContent(title=유니티 뱀서라이크 개발강좌가 온다?! 언데드 서바이버, url=https://www.youtube.com/watch?v=MmW166cHj54, category=test)


In [4]:
# 테스트 오디오 STT
stt_result = stt_strategy.transcribe(f'../downloads/{contents[0].category}/{contents[0].title}.mp3')

/Users/kimhongil/.conda/envs/LumenaAI/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


캐시가 성공적으로 비워졌습니다.


In [5]:
print(type(stt_result))

<class 'dict'>


In [6]:
print(stt_result)

{'text': ' 요즘 너무나 유행하고 있는 이 장르 뱀파이어 서바이버와 비슷하다 뱀서 라이크 그래서 제가 들고 온 타이틀은 바로 언데드 서바이버 자 이 언데드 서바이버 강좌와 함께 여러분들은 유니티 게임 개발에 좀 더 익숙해지면서 뱀서 라이크를 좀 더 이해할 수 있는 그런 시간이 될 겁니다 매직 서바이버 탄탄 특강대 뱀파이어 서바이버 자 모든 개석거라 우리는 언데드 슈 서바이버다 500불도 안되는 적은 코드로 여러분들은 쉽게 따라 하실 수가 있습니다 절대로 약하는게 아니구요 이 강좌를 통해서 여러분들은 2D 기본 움직임에다가 무한이동, 클립팩, 스크립티블 오브젝트 사운드 매니저, 헤드업 디스플레이 조이스틱에 모바일 빌드까지 다채롭게 배우실 수가 있습니다 지금까지 저의 새로운 강좌 언데드 서버에 대한 소개였구요 앞으로 제가 준비한 커리큘럼에 따라서 차근차근 업로드 할테니깐요 많은 기대 바랍니다 감사합니다', 'chunks': [{'timestamp': (1.0, 3.9), 'text': ' 요즘 너무나 유행하고 있는 이 장르'}, {'timestamp': (3.9, 5.5), 'text': ' 뱀파이어 서바이버와 비슷하다'}, {'timestamp': (5.5, 7.0), 'text': ' 뱀서 라이크'}, {'timestamp': (7.0, 9.4), 'text': ' 그래서 제가 들고 온 타이틀은 바로'}, {'timestamp': (9.4, 11.8), 'text': ' 언데드 서바이버'}, {'timestamp': (13.1, 16.1), 'text': ' 자 이 언데드 서바이버 강좌와 함께 여러분들은'}, {'timestamp': (16.1, 19.1), 'text': ' 유니티 게임 개발에 좀 더 익숙해지면서'}, {'timestamp': (19.1, 23.3), 'text': ' 뱀서 라이크를 좀 더 이해할 수 있는 그런 시간이 될 겁니다'}, {'timestamp': (23.3, 25.6), 'text': ' 매직 서바이버 탄탄 특강대 뱀파이어

In [7]:
from domain.value_object import YouTubeScript
from domain.value_object import YouTubeScriptChunk

# 유튜브 스크립트 값 객체
youtube_script = YouTubeScript(
    script=stt_result["text"],
    chunks=[
        YouTubeScriptChunk.from_dict(chunk)
        for chunk in stt_result["chunks"]
    ],
)

In [8]:
print(youtube_script)

YouTubeScript(script=' 요즘 너무나 유행하고 있는 이 장르 뱀파이어 서바이버와 비슷하다 뱀서 라이크 그래서 제가...', chunks=[(1.0-3.9:  요즘 너무나 유행하고 있는...), (3.9-5.5:  뱀파이어 서바이버와 비슷하...), (5.5-7.0:  뱀서 라이크...), ...] (26 chunks))


In [9]:
# DB에 저장
contents[0].set_script(youtube_script)
result = repository.save(contents[0])
print(result)

True


In [10]:
contents = repository.find_without_script()
print(len(contents))

0


In [11]:
# DB 확인
contents = repository.find_all()
print(contents[0].script)

YouTubeScript(script=' 요즘 너무나 유행하고 있는 이 장르 뱀파이어 서바이버와 비슷하다 뱀서 라이크 그래서 제가...', chunks=[(1.0-3.9:  요즘 너무나 유행하고 있는...), (3.9-5.5:  뱀파이어 서바이버와 비슷하...), (5.5-7.0:  뱀서 라이크...), ...] (26 chunks))
